In [13]:
#Importación de los modulos necesarios y lectura del archivo CSV
import pandas as pd
import gmplot
from geopy.geocoders import Nominatim
df = pd.read_csv(r'calles_de_medellin_con_acoso.csv', sep=";")

In [14]:
#Metodo para corregir las coordenadas
def correctFormat(text: str) -> tuple:
    textProcess = text[1:-1]
    partB, partA = textProcess.split(",")
    listReturn = [float(partA),float(partB)]
    return tuple(listReturn)

#Metodo para obtener el nombre de una calle con sus coordenadas
def coordinateToStreet(coordinate: tuple) -> str:
    localizator = Nominatim(user_agent = "SecurityON")
    location = localizator.reverse("{},{}".format(coordinate[0],coordinate[1]))
    return location

#Metodo para sacar el promedio del acoso en toda la ciudad
def average() -> float:
    count = 0
    count2 = 0
    for iteration in df['harassmentRisk']:
        if pd.isnull(iteration):
            continue
        else:
            count2 += 1
            count += iteration
    return count / count2

#Reemplazar los valores NaN en la columna de 'harassmentRisk'
PROMEDIO = average()
df['harassmentRisk'].fillna(PROMEDIO, inplace=True)

In [15]:
#Metodo que crea el grafo
def createGrahp() -> dict:
    uniqueOrigins = pd.unique(df['origin'])
    graph = {}
    for origin in uniqueOrigins:
        correctOrigin = correctFormat(origin)
        graph[correctOrigin] = {}

    for index in df.index:
        originIteration = correctFormat(df['origin'][index])
        destinationIteration = correctFormat(df['destination'][index])
        if destinationIteration in graph:
            if df['oneway'][index]:
                graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] = graph[destinationIteration] | {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
        else:
            if df['oneway'][index]:
                graph[destinationIteration] = {}
                graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}
                graph[destinationIteration] = graph[destinationIteration] | {originIteration:(df['length'][index],df['harassmentRisk'][index])}
            else: graph[originIteration] = graph[originIteration] | {destinationIteration:(df['length'][index],df['harassmentRisk'][index])}

    return graph

graphCM = createGrahp()

In [16]:
#Algoritmo que busca la ruta más corta, retorna una lista en orden con los nodos que representan dicha ruta
def DjikstraFirst(graph: dict, origin: tuple, destination: tuple) -> list:
    shortesdDistance = {}
    previusly = {}
    noVisiteds = graph
    CONTROLER = 999999
    shortestPath = []

    for node in noVisiteds:
        shortesdDistance[node] = CONTROLER
    shortesdDistance[origin] = 0

    while noVisiteds:
        minDistance = None
        for node in noVisiteds:
            if minDistance is None:
                minDistance = node
            elif shortesdDistance[node] < shortesdDistance[minDistance]:
                minDistance = node

        pahtOptions = graph[minDistance].items()

        for childNode, weight in pahtOptions:
            if weight[0] + shortesdDistance[minDistance] < shortesdDistance[childNode]:
                shortesdDistance[childNode] = weight[0] + shortesdDistance[minDistance]
                previusly[childNode] = minDistance

        noVisiteds.pop(minDistance)

    actualyNode = destination

    while actualyNode != origin:
        try:
            shortestPath.insert(0, actualyNode)
            actualyNode = previusly[actualyNode]
        except KeyError:
            print("El camino no es posible de encontrar")
            break

    if shortesdDistance[destination] != CONTROLER:
        #print("La distancia más corta es de: ", str(shortesdDistance))
        print("El camino más corto está dado por: ", str(shortestPath))
        return shortestPath

lista = DjikstraFirst(graphCM,(6.2734442, -75.5443961),(6.2094357, -75.5674348))

El camino más corto está dado por:  [(6.2731964, -75.544468), (6.2729223, -75.5444504), (6.272852, -75.5444377), (6.2728866, -75.5456093), (6.2728127, -75.5456026), (6.2726581, -75.5456187), (6.2724109, -75.5457113), (6.2721471, -75.5457106), (6.272222, -75.5461204), (6.2713431, -75.5463021), (6.2713798, -75.5465003), (6.2698413, -75.5465697), (6.2694021, -75.5469726), (6.2689824, -75.5473578), (6.2685324, -75.5477707), (6.2680583, -75.5482057), (6.2680439, -75.5487727), (6.2680244, -75.5495432), (6.268015, -75.549708), (6.2676405, -75.54966), (6.2672128, -75.5498214), (6.2669347, -75.5504078), (6.2667174, -75.5505451), (6.2661776, -75.5510484), (6.2659428, -75.5511748), (6.2659066, -75.5511811), (6.265331, -75.551242), (6.2647598, -75.5513047), (6.2640163, -75.5524974), (6.263911, -75.5525706), (6.2634704, -75.5530927), (6.262935, -75.553333), (6.2627074, -75.5535577), (6.2621733, -75.5537025), (6.2605274, -75.5532832), (6.2600372, -75.5536752), (6.2596821, -75.5539775), (6.2596493, -

In [17]:
#Genera un archivo HTML con la ruta visualmente
def routhGenerator(paht: list) -> None:
    latitude = [x[0] for x in lista]
    lang = [x[1] for x in lista]
    gmapMedallo = gmplot.GoogleMapPlotter(lista[0][0], lista[0][1], 15)
    gmapMedallo.scatter(latitude, lang, '#F0F8FF', size = 50, marker = False)
    gmapMedallo.plot(latitude, lang, 'aliceblue', edge_width = 2.5)
    gmapMedallo.draw("medallo.html")

routhGenerator(lista)